### Notes
We might need to remove the common words in the corpus beside the stop words as those are common cross all corpus

In [ ]:
import pandas as pd
import nltk as nltk
from cleanup.utterances import custom_tokenize
import re

utterances_df = pd.read_csv('./temp_data/utterances.csv')
utterances_df.set_index('id')
utterances_df['actionbody'] = utterances_df['actionbody'].astype('str')
print(len(utterances_df))
utterances_df.head(2)

In [ ]:
uttr_voc_df1 = pd.read_csv('./temp_data/utterances_voc_20250316.csv')
uttr_voc_df1.set_index('word',inplace=True)
uttr_voc_df1 = uttr_voc_df1.dropna()
print(len(uttr_voc_df1))
uttr_voc_df1.head(5)

In [ ]:
uttr_voc_df2 = pd.read_csv('temp_data/voc/vocabulary_20250322.csv')
uttr_voc_df2.set_index('word',inplace=True)
uttr_voc_df2 = uttr_voc_df2.dropna()
print(len(uttr_voc_df2))
uttr_voc_df2.head(5)

In [ ]:
word_mapping_df = pd.concat([uttr_voc_df1,uttr_voc_df2])
print(len(word_mapping_df))
word_mapping_df.head(3)

In [ ]:
rows = utterances_df.iterrows()
utterances_df['changed'] = False
utterances_df['actionbody_old'] = utterances_df['actionbody']
version_regex = re.compile(r'\b[vV][\d\.]+\b')
for idx,r in rows:
    body = r['actionbody']
    body = re.sub(version_regex, 'ph_version', body)
    body_words = custom_tokenize(body)
    updated_body = ""
    affected = False
    ph_tec_count = 0
    for word in body_words:
        w = word
        if word in word_mapping_df.index:
            w = word_mapping_df.loc[w,'type']
            if w == 'ph_technical':
                ph_tec_count = ph_tec_count + 1
            affected = True
        updated_body += w  
    if ph_tec_count / (len(body_words)/2) > 0.4:
        updated_body = 'ph_technical'
    if affected:
        utterances_df.loc[idx,'actionbody'] = updated_body
        utterances_df.loc[idx,'changed'] = True

print('done')

In [ ]:
changed_df = utterances_df[utterances_df['changed'] == True]
changed_df[['id','issueid', 'created', 'author', 'utr_seq',
       'is_private', 'comment_seq','actionbody_old', 'actionbody']].set_index('id').to_csv('./temp_data/utterances_cleaned_changed.csv')
print(f'{len(changed_df)} of {len(utterances_df)}')

In [ ]:
unchanged_df = utterances_df[utterances_df['changed'] == False]
unchanged_df[['id','issueid', 'created', 'author', 'utr_seq',
       'is_private', 'comment_seq','actionbody']].set_index('id').to_csv('./temp_data/utterances_cleaned.csv')
print('done')        